In [1]:
%cd /content
!rm -rf Gradu
!git clone https://github.com/Elkkujou/Gradu.git
%cd /content/Gradu
!ls

/content
Cloning into 'Gradu'...
remote: Enumerating objects: 316, done.
remote: Counting objects: 100% (84/84), done.
remote: Compressing objects: 100% (41/41), done.
remote: Total 316 (delta 64), reused 43 (delta 43), pack-reused 232 (from 2)
Receiving objects: 100% (316/316), 44.13 MiB | 26.85 MiB/s, done.
Resolving deltas: 100% (145/145), done.
/content/Gradu
 chatti_RF.ipynb	       MSCI_XGBOOST.ipynb		    RF_Gradu.ipynb
 Fama_french_XGBOOST.ipynb     Regiimi_prediction.ipynb		   'RF REGIIMI HYVÄ TRAINING.ipynb'
'Financial turbulence.ipynb'   regime_prediction_famafrench.ipynb  'RF_regime (3).ipynb'
 FT_source.xlsx		       regime_prediction_msci.ipynb	    THE_ONE.xlsx
 Gradient_boost_malli.ipynb    regime_pred.txt


In [2]:
calc_drawdowns = True
drawdowns_from = 'Mkt-RF'
cluster_n = 2

DL = False
ML = True

In [3]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns

from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.metrics import confusion_matrix, accuracy_score
from sklearn.neighbors import KNeighborsClassifier
from sklearn.preprocessing import StandardScaler

import statsmodels.api as sm
from tabulate import tabulate

from IPython.display import display, HTML
from sklearn.cluster import KMeans

In [4]:
xls_file = pd.ExcelFile("/content/Gradu/THE_ONE.xlsx")
excel_df = xls_file.parse("22.2.2025")

excel_df.columns = excel_df.columns.get_level_values(0)
excel_df['Date'] = pd.to_datetime(excel_df['Date'])
excel_df.sort_values('Date', inplace=True)

print("Headers in the 'returns non-log' sheet:")
print(excel_df.columns)

Headers in the 'returns non-log' sheet:
Index(['Date', 'Mkt-RF', 'SMB', 'HML', 'RMW', 'CMA', 'RF', 'SMB-12A',
       'HML-12A', 'RMW-12A', 'CMA-12A', 'RSI-SMB', 'RSI-HML', 'RSI-RMW',
       'RSI-CMA', 'CPI', 'Cape', 'Treasury_10Y', 'Winning Factor', 'check',
       'NFCI', 'T10YFF', 'DGS1', 'DGS5', 'GARCH'],
      dtype='object')


In [5]:
if calc_drawdowns:
    # Calculate the cumulative portfolio values using excel_df and the specified drawdowns_from column
    portfolio = (1 + excel_df[drawdowns_from]).cumprod()

    # Define a function to compute the maximum drawdown over a rolling window
    def max_dd(s):
        return (s / s.cummax() - 1).min()

    # Calculate the 3-month rolling drawdown using the portfolio series
    dd_series = portfolio.rolling(window=3, min_periods=3).apply(max_dd, raw=False)

    # Create a temporary DataFrame to store dates and the calculated drawdowns
    dd_temp = pd.DataFrame({
        'Date': excel_df['Date'],
        '3M_Max_Drawdown': dd_series
    })

    # Drop rows with missing drawdown data to ensure complete observations
    dd_temp = dd_temp.dropna()

    print("Drawdown calculation (temporary DataFrame):")
    print(dd_temp.head())


Drawdown calculation (temporary DataFrame):
        Date  3M_Max_Drawdown
2 1963-10-01        -0.015728
3 1963-11-01        -0.015728
4 1963-12-01        -0.008481
5 1964-01-01        -0.008481
6 1964-02-01         0.000000


In [6]:
# Fit KMeans clustering on the temporary DataFrame's '3M_Max_Drawdown' column
kmeans = KMeans(n_clusters=cluster_n, random_state=42)
dd_temp['DD_cluster'] = kmeans.fit_predict(dd_temp[['3M_Max_Drawdown']])

# Merge the cluster labels from dd_temp into excel_df by matching on the 'Date' column.
# This adds a new column 'DD_cluster' to excel_df.
excel_df = excel_df.merge(dd_temp[['Date', 'DD_cluster']], on='Date', how='left')

print("excel_df with DD_cluster added:")
from IPython.display import display, HTML
display(HTML(excel_df.head().to_html(index=False)))


excel_df with DD_cluster added:


Date,Mkt-RF,SMB,HML,RMW,CMA,RF,SMB-12A,HML-12A,RMW-12A,CMA-12A,RSI-SMB,RSI-HML,RSI-RMW,RSI-CMA,CPI,Cape,Treasury_10Y,Winning Factor,check,NFCI,T10YFF,DGS1,DGS5,GARCH,DD_cluster
1963-08-01,-0.004027,-0.004313,-0.009794,0.005491,-0.013051,0.002643,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,30.69,19.969232,4.00,RMW,0,NaN,0.52,3.59,3.93,0.004094,NaN
1963-09-01,0.050801,-0.007409,0.016690,0.002689,-0.000824,0.002423,NaN,NaN,NaN,NaN,50.855086,51.525905,51.059536,50.000000,30.75,20.472638,4.08,HML,0,NaN,0.57,3.59,3.94,0.003689,NaN
1963-10-01,-0.015728,-0.004244,0.000064,-0.005911,0.002676,0.002804,NaN,NaN,NaN,NaN,50.311665,49.889949,50.099010,49.924242,30.72,20.960360,4.11,CMA,1,NaN,0.65,3.68,4.01,0.005001,1.0
1963-11-01,0.025101,-0.013001,0.000175,0.025746,-0.015377,0.002994,NaN,NaN,NaN,NaN,52.384292,50.151057,48.916117,50.187032,30.75,20.891345,4.12,RMW,0,NaN,0.58,3.74,3.99,0.004213,1.0
1963-12-01,-0.008481,-0.010032,0.018485,-0.004858,0.021257,0.002703,NaN,NaN,NaN,NaN,48.692265,49.473684,51.188052,49.010198,30.78,20.720399,4.13,CMA,0,NaN,0.89,3.83,4.06,0.013948,1.0


In [9]:

FEATURES = [
    'CPI', 'Cape', 'Treasury_10Y', 'GARCH'
]

TRUE_REGIME = 'DD_cluster'
DATE = 'Date'

selected_columns =  [DATE] + [TRUE_REGIME] +FEATURES


In [10]:
missing = [col for col in selected_columns if col not in excel_df.columns]
if missing:
    raise ValueError(f"Missing columns in df: {missing}")

# Subset and drop rows with empty values
input_df = excel_df[selected_columns]
orig_rows = len(input_df)
input_df = input_df.dropna()
print(f"Original rows: {orig_rows}, Dropped rows: {orig_rows - len(input_df)}")
print(f"First obs: {input_df[DATE].min()}, Last obs: {input_df[DATE].max()}")

# Display as neat HTML table (adjust 'rows_to_show' as needed)
rows_to_show = 10
from IPython.display import display, HTML
display(HTML(input_df.head(rows_to_show).to_html(index=False)))

Original rows: 738, Dropped rows: 2
First obs: 1963-10-01 00:00:00, Last obs: 2025-01-01 00:00:00


Date,DD_cluster,CPI,Cape,Treasury_10Y,GARCH
1963-10-01,1.0,30.72,20.960360,4.11,0.005001
1963-11-01,1.0,30.75,20.891345,4.12,0.004213
1963-12-01,1.0,30.78,20.720399,4.13,0.013948
1964-01-01,1.0,30.88,21.038599,4.17,0.003212
1964-02-01,1.0,30.94,21.627216,4.15,0.002518
1964-03-01,1.0,30.91,21.832671,4.22,0.002122
1964-04-01,1.0,30.94,22.167246,4.23,0.002052
1964-05-01,1.0,30.95,22.422192,4.20,0.003026
1964-06-01,1.0,30.98,22.574331,4.17,0.002220
1964-07-01,1.0,31.01,22.300288,4.19,0.002613


In [11]:
from tabulate import tabulate

# Calculate the number of observations for each regime in the data
regime_counts = input_df[TRUE_REGIME].value_counts().sort_index().reset_index()
regime_counts.columns = [TRUE_REGIME, 'Count']

print("Number of observations per regime:")
print(tabulate(regime_counts, headers='keys', tablefmt='psql', showindex=False))


Number of observations per regime:
+--------------+---------+
|   DD_cluster |   Count |
|--------------+---------|
|            0 |     144 |
|            1 |     592 |
+--------------+---------+


In [12]:
if ML == True:

    import numpy as np
    import matplotlib.pyplot as plt
    import seaborn as sns
    import pandas as pd
    from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
    from sklearn.svm import SVC
    from sklearn.naive_bayes import GaussianNB
    from sklearn.linear_model import LogisticRegression
    from sklearn.metrics import accuracy_score, confusion_matrix

    # Define predictors and target using the given column lists
    predictors = FEATURES
    target = TRUE_REGIME

    # Drop rows with missing values in predictors or target
    data = input_df.dropna(subset=predictors + [target]).reset_index(drop=True)
    X = data[predictors]
    y = data[target]
    # Also grab the DATE column for printing prediction timestamps
    dates = data[DATE]

    # Define the base models with high parameters, now including Gradient Boosting
    base_models = {
        'RandomForest': RandomForestClassifier(n_estimators=200, random_state=42),
        'GradientBoosting': GradientBoostingClassifier(n_estimators=200, learning_rate=0.1, random_state=42),
        'SVC': SVC(probability=True, C=1000, kernel='rbf', gamma='scale'),
        'GaussianNB': GaussianNB(var_smoothing=1e-8)
    }

    # Define the meta-learner
    meta_learner = LogisticRegression(max_iter=1000, random_state=42)

    window_size = 60  # fixed window length once reached
    min_data_pct = 0.4  # start base model predictions when available data >= 80% of window_size
    base_threshold = int(window_size * min_data_pct)

    # Initialize dictionaries/lists to store rolling accuracies and meta weights
    rolling_accuracies_base = {name: [] for name in base_models}
    rolling_accuracies_meta = []
    meta_weights_list = []  # record meta-learner coefficients

    # Initialize expanding meta training set (meta features and outcomes)
    meta_train_features = []
    meta_train_targets = []

    print("Starting rolling window evaluation with stacking ensemble and expanding meta learner:")

    for i in range(len(X)):
        # Do not make any prediction until we have at least base_threshold observations
        if i < base_threshold:
            print(f"Skipping prediction for timestamp {dates.iloc[i]} "
                  f"(insufficient data: {i} observations, need {base_threshold}).")
            continue

        # For base models, if i is less than full window_size, use all data from start to i.
        # Once i >= window_size, use a fixed rolling window of size window_size.
        if i < window_size:
            X_train_window = X.iloc[:i]
            y_train_window = y.iloc[:i]
        else:
            X_train_window = X.iloc[i-window_size:i]
            y_train_window = y.iloc[i-window_size:i]

        X_test_window = X.iloc[[i]]
        y_test_window = y.iloc[[i]]
        test_date = dates.iloc[i]

        # --- Step 1: Generate meta features from base model predictions ---
        meta_features_test = np.zeros((1, len(base_models)))
        for j, (name, model) in enumerate(base_models.items()):
            model.fit(X_train_window, y_train_window)
            probs = model.predict_proba(X_test_window)
            if probs.shape[1] == 1:
                # Edge-case: model returns only one class probability
                y_pred_prob = 1.0 if model.classes_[0] == 1 else 0.0
            else:
                y_pred_prob = probs[0, 1]
            meta_features_test[0, j] = y_pred_prob

            # Record accuracy for each base model on this test observation
            y_pred = model.predict(X_test_window)[0]
            acc = accuracy_score(y_test_window, [y_pred])
            rolling_accuracies_base[name].append(acc)

        # --- Step 2: Use meta training set to make meta prediction once full window is reached ---
        if i >= window_size:
            meta_learner.fit(np.array(meta_train_features), np.array(meta_train_targets))
            meta_pred = meta_learner.predict(meta_features_test)[0]
            meta_acc = accuracy_score(y_test_window, [meta_pred])
            rolling_accuracies_meta.append(meta_acc)
            meta_weights_list.append(meta_learner.coef_[0])
            prediction_source = "Meta Learner"
            print(f"Test date {test_date}: {prediction_source} Prediction = {meta_pred}, Accuracy = {meta_acc:.2f}")
        else:
            # For i in [base_threshold, window_size), only base model predictions are available
            print(f"Test date {test_date}: Base Model predictions only (growing training window).")

        # --- Step 3: Append current meta features and true outcome to the expanding meta training set ---
        meta_train_features.append(meta_features_test[0])
        meta_train_targets.append(y_test_window.iloc[0])

    # --- Aggregation of Results ---
    avg_base_acc = {name: np.mean(acc_list) if acc_list else 0 for name, acc_list in rolling_accuracies_base.items()}
    avg_meta_acc = np.mean(rolling_accuracies_meta) if rolling_accuracies_meta else 0
    avg_meta_weights = np.mean(meta_weights_list, axis=0) if meta_weights_list else np.array([])

    results_table = pd.DataFrame({
        'Model': list(avg_base_acc.keys()) + ['Meta Learner'],
        'Average Accuracy': list(avg_base_acc.values()) + [avg_meta_acc]
    })
    print("\nRolling Window Average Accuracies:")
    print(results_table)

    if avg_meta_weights.size > 0:
        weights_table = pd.DataFrame({
            'Base Model': list(base_models.keys()),
            'Average Weight': avg_meta_weights
        })
        print("\nAggregate Meta-Learner Weights (Average Coefficients):")
        print(weights_table)


Starting rolling window evaluation with stacking ensemble and expanding meta learner:
Skipping prediction for timestamp 1963-10-01 00:00:00 (insufficient data: 0 observations, need 24).
Skipping prediction for timestamp 1963-11-01 00:00:00 (insufficient data: 1 observations, need 24).
Skipping prediction for timestamp 1963-12-01 00:00:00 (insufficient data: 2 observations, need 24).
Skipping prediction for timestamp 1964-01-01 00:00:00 (insufficient data: 3 observations, need 24).
Skipping prediction for timestamp 1964-02-01 00:00:00 (insufficient data: 4 observations, need 24).
Skipping prediction for timestamp 1964-03-01 00:00:00 (insufficient data: 5 observations, need 24).
Skipping prediction for timestamp 1964-04-01 00:00:00 (insufficient data: 6 observations, need 24).
Skipping prediction for timestamp 1964-05-01 00:00:00 (insufficient data: 7 observations, need 24).
Skipping prediction for timestamp 1964-06-01 00:00:00 (insufficient data: 8 observations, need 24).
Skipping predi

In [ ]:
if DL == True:
    import tensorflow as tf
    from tensorflow.keras.models import Sequential, Model
    from tensorflow.keras.layers import Input, LSTM, Dense, Dropout
    import xgboost as xgb
    from sklearn.metrics import confusion_matrix, accuracy_score
    import matplotlib.pyplot as plt
    import seaborn as sns
    import numpy as np

    # Parameters:
    training_window = 120  # Use the previous 120 months as training data
    seq_len = 60           # Generate overlapping sequences of 60 months within the training window
    num_classes = cluster_n  # Regimes are assumed to be labeled 0, 1, 2
    num_features = len(FEATURES)

    all_preds = []   # To store predictions
    all_true = []    # To store true regime labels
    dates_list = []  # To store corresponding dates

    # Ensure data is sorted by date
    data = input_df.sort_values(DATE).reset_index(drop=True)

    # Rolling window: start at index = training_window until end of data
    for i in range(training_window, len(data)):
        # Use the previous 120 months as the training window
        train_window = data.iloc[i - training_window:i].reset_index(drop=True)
        if len(train_window) < training_window:
            continue

        # Generate overlapping sequences of length 60 within the training window
        X_train_sequences = []
        y_train = []
        for j in range(len(train_window) - seq_len + 1):
            seq = train_window[FEATURES].iloc[j : j + seq_len].values
            target = train_window[TRUE_REGIME].iloc[j + seq_len - 1]  # target is the regime at the end of the sequence
            X_train_sequences.append(seq)
            y_train.append(target)
        X_train_sequences = np.array(X_train_sequences)  # shape: (num_samples, seq_len, num_features)
        y_train = np.array(y_train)

        # Build an LSTM feature extractor model using an Input layer
        input_layer = Input(shape=(seq_len, num_features))
        lstm_out = LSTM(32, return_sequences=False)(input_layer)
        dropout_out = Dropout(0.2)(lstm_out)
        feature_extractor = Model(inputs=input_layer, outputs=dropout_out)

        # Extract features from the training sequences
        X_train_features = feature_extractor.predict(X_train_sequences, verbose=0)

        # Train an XGBoost classifier on these features.
        clf = xgb.XGBClassifier(n_estimators=50, use_label_encoder=False, eval_metric='mlogloss')
        clf.fit(X_train_features, y_train)

        # For prediction, use the last 60-month sequence in the training window
        X_pred_seq = train_window[FEATURES].iloc[-seq_len:].values.reshape(1, seq_len, num_features)
        X_pred_feature = feature_extractor.predict(X_pred_seq, verbose=0)
        pred_regime = clf.predict(X_pred_feature)[0]

        # The true regime for the next month (time index i)
        true_regime = data[TRUE_REGIME].iloc[i]

        all_preds.append(pred_regime)
        all_true.append(true_regime)
        dates_list.append(data[DATE].iloc[i])

        print(f"Prediction for {data[DATE].iloc[i]}: Predicted={pred_regime}, True={true_regime}")

    # Compute overall accuracy
    accuracy = accuracy_score(all_true, all_preds)
    print(f"\nOverall Accuracy: {accuracy*100:.2f}%")

    # Create and visualize the confusion matrix
    cm = confusion_matrix(all_true, all_preds)
    plt.figure(figsize=(6, 4))
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues')
    plt.title("Rolling Window Hybrid (LSTM + XGBoost) Confusion Matrix")
    plt.xlabel("Predicted")
    plt.ylabel("True")
    plt.show()
